In [1]:
## data cleaning and exploration
## data exploration/ exploratory analysis - univariate, multi variate, PCA
## build model
## output

## improvise:
## use exploration results
## visualize exploration results
## feature selection
## missing value replacement

## problems to tackle:
## feature selection: bmi can be dropped (find independent variables)
## class imbalance between accepted and denied - overfitting/ underfitting

### Problem Apparoach
##### Step 1
1. data cleaning and exploration
2. data exploration - exploratory analysis - univariate, multi variate, PCA
3. build model
4. output

##### Step 2 - Improvise
1. Use exploration resuslts
2. visualize exploration results
3. feature selection
4. missing value replacement

##### Problems to tackle
1. feature selection: bmi can be dropped (find independent variables)
2. overfitting/ underfitting (class imbalance)


##### Thoughts
1. Pick accepted and denied classes, rest no need for training
2. Some BMI values are incorrect assuming height and weight are correct. Height in cm, weight in kg
3. Maybe create new features from the existing ones?
4. BMI formula: bmi = w / ((h/100)*(h/100))
5. for visualizations, get rid of Nans

In [24]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import openpyxl
import speedml

In [16]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 1000)

In [17]:
## read csv file, format columns, remove unnecessary column
df = pd.read_csv('TestData_v2.csv')
df.columns = df.columns.str.replace(' ', '')
df.drop(['Unnamed:0'], axis=1, inplace=True)
# df.PatientID = df.PatientID.astype('int')

In [ ]:
df.PatientID = df.PatientID.astype('int')

In [18]:
## drop rows with all Nan values
df.dropna(how='all', inplace=True)
df.reset_index(drop=True, inplace=True)

In [19]:
##convert categorical variables into string or category codes - str or int 
## all object dtypes into str
df.Label = df.Label.astype('category')
# df['LabelID'] = df.Label.cat.codes

In [20]:
cat_columns = ['FourKHz', 'OneKHz', 'UrineProtein', 'UrineSugar', 'Urobilinogen', 'OccultBlood', 'Ketones', 'HBAntigen',
               'TPHA', 'HCVAntibody']

for col in cat_columns:
    df[col] = df[col].astype('category')    
#     print(col, dict(enumerate(df[col].cat.categories)))
#     df[col] = df[col].cat.codes
    
#     df[str(col)] = pd.Categorical(df[str(col)])
#     df[str(col)] = df[str(col)].cat.codes
    
#     key_map = dict(enumerate(df[col].cat.categories))
#     key_map.update({0: 'zero', 1: 'one'})
#     print(key_map)
#     df.replace({col: key_map}, inplace=True)
#     df['mycol_codes'] = df['mycol'].cat.codes
#     df['mycol_reversed'] = df['mycol_codes'].map(d)

In [9]:
## calculate or update BMI
# df['BMI'] = df['Weight'] / ((df['Height']/100) * (df['Height']/100))
df['BMI'] = df.apply(
    lambda row: row['Weight']/((row['Height']/100)*(row['Height']/100)) if np.isnan(row['BMI']) else row['BMI'],
    axis=1
)

# df['Height'] = df.apply(
#     lambda row: row['Height']*row['Weight'] if np.isnan(row['BMI']) else row['BMI'],
#     axis=1
# )

In [10]:
df.groupby('Label').describe()

Height                                                                \
          count        mean       std    min      25%     50%      75%    max   
Label                                                                           
Accepted   99.0  169.282828  8.882051  119.0  166.500  170.00  173.100  190.0   
Denied     32.0  169.362500  7.773352  157.2  161.875  170.00  175.025  183.6   
Pending    22.0  171.331818  9.271995  148.6  164.500  171.55  178.175  184.9   
Referred   38.0  168.892105  7.782877  153.4  163.125  168.00  175.075  187.5   

         Weight                                                               \
          count       mean        std   min     25%    50%       75%     max   
Label                                                                          
Accepted   99.0  70.705354  45.404214  22.0  57.800  64.00  72.30000  480.00   
Denied     24.0  83.179792  67.457047  22.0  54.625  65.85  85.12875  373.00   
Pending    23.0  68.447826  24.281509  20.0  58.300  66.80  88.85000  110.50   
Referred   42.0  76.381024  44.335378  25.0  55.950  66.00  83.60000  311.53   

            BMI                                                           \
          count       mean       std        min        25%   50%     75%   
Label                                                                      
Accepted  115.0  23.813621  5.908577   8.928571  21.500000  22.6  24.950   
Denied     41.0  23.825852  5.398528  17.200000  20.100000  22.5  26.000   
Pending    23.0  26.659397  8.294281  17.400000  21.782272  24.7  29.250   
Referred   52.0  26.824710  8.860885  16.000000  22.375985  24.9  28.025   

                    AbdominalCircumference                              \
                max                  count       mean        std   min   
Label                                                                    
Accepted  57.803468                   69.0  83.168261  16.503291  39.0   
Denied    39.843750                   19.0  87.021053  17.492684  35.3   
Pending   57.000000                   15.0  81.486667  14.743854  35.0   
Referred  58.000000                   24.0  83.562500  14.848915  45.4   

                                      VitalCapacity                           \
             25%   50%     75%    max         count      mean       std  min   
Label                                                                          
Accepted  81.000  84.9  85.000  185.0          19.0  2.731053  1.622782  1.0   
Denied    83.450  85.0  91.500  130.0          17.0  2.967059  2.063067  1.0   
Pending   82.500  84.9  85.000  103.8           6.0  2.966667  1.544884  1.0   
Referred  78.125  84.9  86.725  118.0          15.0  3.996000  2.381931  1.0   

                                  BloodPressureSecond                        \
           25%   50%    75%   max               count       mean        std   
Label                                                                         
Accepted  1.00  3.00  4.120  5.15                48.0  79.368750  17.298237   
Denied    1.00  2.64  4.280  7.00                11.0  73.454545   8.201995   
Pending   2.00  2.75  4.100  5.00                11.0  77.454545  18.581516   
Referred  2.76  4.00  4.025  8.00                21.0  73.571429  18.500193   

                                          BloodPressureFirst              ...  \
           min    25%   50%    75%    max              count        mean  ...   
Label                                                                     ...   
Accepted  31.7  69.75  80.0  84.25  147.0               59.0  113.374576  ...   
Denied    57.0  69.50  76.0  79.50   84.0               18.0  108.911111  ...   
Pending   28.0  75.50  84.0  84.50  102.0               11.0  118.909091  ...   
Referred  21.0  68.00  72.0  84.00  110.0               28.0  109.035714  ...   

         bodyFatPercentage         Lym                                     \
                       75%   max count       mean        std   min  

In [11]:
df.describe()

,Height,Weight,BMI,AbdominalCircumference,VitalCapacity,BloodPressureSecond,BloodPressureFirst,UrinePH,Leukocyte,RedBloodCells,Hemoglobin,Hematocrit,MCV,MCH,MCHC,platelet,GOT,albumin,GPT,GammaGPT,LDH,ALP,Bilirubin,UreaNitrogen,creatinine,eGFR,HDL,NeutralFat,LDL,TotalProtein,AGRatio,Amylase,UricAcid,CRP,HbA1c,BloodSugar,FastingBloodSugar,bodyFatPercentage,Lym,TTT,RightEyeCorrection,Prostate,LeftEyeCorrection,PatientID
count,191.000000,188.000000,231.000000,127.000000,57.000000,91.000000,116.000000,133.000000,91.000000,195.000000,208.000000,199.000000,138.000000,159.000000,158.000000,159.000000,401.000000,103.000000,321.000000,360.000000,91.000000,143.000000,139.000000,59.000000,229.000000,95.000000,281.000000,207.000000,248.000000,76.000000,56.000000,61.000000,165.000000,69.000000,130.000000,145.000000,85.000000,59.000000,77.000000,51.000000,93.000000,108.000000,94.000000,9.970000e+02
mean,169.454450,73.289617,24.776959,83.620551,3.159123,77.084615,112.159483,5.385602,6003.021978,437.353846,13.080865,43.772362,92.300725,31.862453,32.899873,27.563711,22.121653,12.098447,24.928956,37.244112,185.705604,210.420280,1.111168,13.705085,0.878421,76.951684,65.876584,116.811594,114.972738,6.941316,1.666964,67.704918,4.717455,0.154797,5.523846,99.162759,98.524706,20.091695,24.645455,3.056863,2.026882,1.069843,2.186170,4.018053e+06
std,8.505043,46.492055,6.949127,16.046830,1.989713,16.895245,29.162597,2.156957,2791.484862,169.228763,4.338348,6.029684,16.551911,9.172809,6.441854,15.700692,15.599028,19.920741,15.762314,31.149123,59.873190,75.310282,0.866502,5.573318,0.358824,14.230130,30.962630,74.589767,38.810800,1.187812,0.708955,26.647291,2.798277,0.159749,1.264905,30.477681,25.262375,8.998808,14.492244,2.241183,2.277114,0.691358,2.867939,1.521438e+04
min,119.000000,20.000000,8.928571,35.000000,1.000000,21.000000,33.000000,1.100000,100.000000,52.000000,4.700000,26.400000,21.000000,12.000000,11.100000,10.400000,1.000000,1.000000,3.000000,1.000000,31.000000,21.000000,0.200000,3.000000,0.020000,50.300000,21.000000,23.000000,37.000000,3.000000,0.550000,11.000000,1.000000,0.010000,1.990000,30.000000,28.000000,3.000000,8.100000,1.000000,0.100000,0.005000,0.100000,3.956543e+06
25%,164.600000,57.375000,21.433138,81.000000,1.000000,69.000000,100.750000,4.000000,4845.000000,435.000000,11.425000,40.200000,87.550000,29.000000,31.525000,19.050000,10.000000,4.000000,13.000000,15.000000,157.500000,160.000000,0.615000,10.000000,0.700000,66.450000,45.000000,60.500000,86.750000,6.700000,1.075000,49.000000,1.000000,0.040000,5.000000,86.000000,87.600000,14.950000,11.000000,1.000000,0.900000,0.619750,0.700000,4.012759e+06
50%,169.800000,64.950000,23.500000,84.900000,3.500000,80.000000,121.000000,5.000000,6100.000000,479.000000,13.800000,44.400000,91.000000,31.100000,33.000000,24.600000,21.000000,4.400000,21.000000,28.500000,177.000000,206.000000,1.000000,13.700000,0.830000,77.000000,57.000000,101.000000,116.500000,7.100000,1.600000,64.000000,5.400000,0.100000,5.400000,93.000000,96.000000,20.800000,20.600000,2.500000,1.000000,1.000000,1.000000,4.015578e+06
75%,174.600000,75.050000,25.750000,85.255000,4.220000,84.000000,130.000000,6.000000,7235.000000,521.500000,15.200000,46.600000,95.000000,32.850000,34.000000,30.150000,30.000000,4.900000,31.000000,50.000000,208.500000,262.500000,1.100000,17.000000,1.000000,86.350000,75.000000,151.000000,135.000000,7.425000,1.892500,79.000000,6.600000,0.210000,5.800000,102.000000,102.000000,26.400000,36.100000,4.000000,2.000000,1.047500,1.875000,4.032453e+06
max,190.000000,480.000000,58.000000,185.000000,8.000000,147.000000,174.000000,15.000000,18370.000000,1422.000000,33.900000,81.000000,219.000000,101.000000,94.000000,85.000000,97.000000,96.000000,80.000000,149.000000,559.000000,397.000000,5.000000,32.100000,3.000000,111.110000,193.000000,389.000000,254.000000,11.000000,4.000000,134.000000,15.500000,0.700000,10.000000,336.000000,184.000000,53.000000,71.000000,11.000000,13.000000,3.000000,12.000000,4.037800e+06


In [12]:
accepted = df[df['Label'] == 'Accepted']
denied = df[df['Label'] == 'Denied']
pending = df[df['Label'] == 'Pending']
referred = df[df['Label'] == 'Referred']

In [32]:
accepted_per = round(len(accepted.Label)/(len(accepted.Label)+len(denied.Label)), 2)
denied_per = round(len(denied.Label)/(len(accepted.Label)+len(denied.Label)), 2)
# denied_per = denied.count()/(accepted.count()+denied.count())

In [33]:
print(accepted_per)
print(denied_per)

## classes are imbalanced

0.72
0.28


In [34]:
df.groupby('Label').mean()

,Height,Weight,BMI,AbdominalCircumference,VitalCapacity,BloodPressureSecond,BloodPressureFirst,UrinePH,Leukocyte,RedBloodCells,Hemoglobin,Hematocrit,MCV,MCH,MCHC,platelet,GOT,albumin,GPT,GammaGPT,LDH,ALP,Bilirubin,UreaNitrogen,creatinine,eGFR,HDL,NeutralFat,LDL,TotalProtein,AGRatio,Amylase,UricAcid,CRP,HbA1c,BloodSugar,FastingBloodSugar,bodyFatPercentage,Lym,TTT,RightEyeCorrection,Prostate,LeftEyeCorrection,PatientID
Label,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Accepted,169.282828,70.705354,23.592532,83.168261,2.731053,79.368750,113.374576,5.181635,6043.183673,457.244186,13.584896,44.312000,93.100000,30.962712,33.518182,28.733000,20.367756,11.802632,23.794480,35.289154,172.874516,219.672131,1.063475,14.893939,0.887789,78.006176,65.527923,105.455000,111.786296,6.739118,1.494000,69.545455,4.973165,0.144167,5.405849,94.171642,100.058824,21.550278,23.603448,3.139130,1.984314,1.251382,1.733333,4.018137e+06
Denied,169.362500,83.179792,22.978788,87.021053,2.967059,73.454545,108.911111,5.539286,6040.000000,390.916667,12.064706,42.741667,93.107143,34.812121,31.366061,28.157576,25.048718,10.232069,21.265625,35.322507,182.571429,202.244828,1.317667,14.975000,0.789574,75.422222,68.115385,119.615152,119.727273,6.800000,1.621333,71.666667,4.373333,0.105947,6.011053,105.826087,98.873684,17.880000,22.786667,2.055556,2.128571,1.063000,1.957143,4.020214e+06
Pending,171.331818,68.447826,28.187333,81.486667,2.966667,77.454545,118.909091,4.768889,3622.000000,425.520000,13.457143,43.117500,88.788889,32.469091,32.663636,29.044375,20.610714,14.541667,26.695556,39.631951,200.583333,226.078571,1.058095,11.687500,0.970079,79.300000,66.875000,106.934483,115.774194,7.436364,1.571429,59.285714,4.287500,0.263750,5.839474,95.631579,99.000000,15.025000,32.510000,2.200000,1.616667,1.132900,4.420000,4.017100e+06
Referred,168.892105,76.381024,27.900000,83.562500,3.996000,73.571429,109.035714,5.882857,7000.052632,442.708333,12.593400,43.750208,92.277500,30.582444,33.354167,24.435250,23.646465,13.600417,28.340057,40.470297,196.262963,196.407692,1.036706,11.692857,0.889241,75.884375,64.338657,146.357778,116.666446,7.122778,2.010714,65.294118,4.642500,0.172111,5.293077,106.058333,95.105263,18.966667,23.752174,3.871429,2.172727,0.843517,2.394737,4.016741e+06


In [13]:
accepted.describe()

,Height,Weight,BMI,AbdominalCircumference,VitalCapacity,BloodPressureSecond,BloodPressureFirst,UrinePH,Leukocyte,RedBloodCells,Hemoglobin,Hematocrit,MCV,MCH,MCHC,platelet,GOT,albumin,GPT,GammaGPT,LDH,ALP,Bilirubin,UreaNitrogen,creatinine,eGFR,HDL,NeutralFat,LDL,TotalProtein,AGRatio,Amylase,UricAcid,CRP,HbA1c,BloodSugar,FastingBloodSugar,bodyFatPercentage,Lym,TTT,RightEyeCorrection,Prostate,LeftEyeCorrection,PatientID
count,99.000000,99.000000,115.000000,69.000000,19.000000,48.000000,59.000000,52.000000,49.000000,86.000000,96.000000,95.00000,52.000000,59.000000,55.000000,70.000000,168.000000,38.000000,125.000000,145.000000,31.000000,61.000000,55.000000,33.000000,95.000000,34.000000,130.000000,100.000000,108.000000,34.000000,20.000000,22.000000,79.000000,24.000000,53.000000,67.000000,34.000000,36.000000,29.000000,23.000000,51.000000,34.000000,51.000000,4.580000e+02
mean,169.282828,70.705354,23.813621,83.168261,2.731053,79.368750,113.374576,5.181635,6043.183673,457.244186,13.584896,44.31200,93.100000,30.962712,33.518182,28.733000,20.367756,11.802632,23.794480,35.289154,172.874516,219.672131,1.063475,14.893939,0.887789,78.006176,65.527923,105.455000,111.786296,6.739118,1.494000,69.545455,4.973165,0.144167,5.405849,94.171642,100.058824,21.550278,23.603448,3.139130,1.984314,1.251382,1.733333,4.018137e+06
std,8.882051,45.404214,5.908577,16.503291,1.622782,17.298237,27.682103,2.146239,2289.372910,172.065236,3.967447,5.77996,21.098704,1.974658,5.287970,14.637965,13.012972,19.268817,15.270821,30.393297,36.845599,73.328421,0.865345,5.245709,0.340766,13.115132,30.739426,65.351621,33.936629,1.259231,0.459363,24.786185,2.617015,0.153353,0.869618,19.638504,23.769016,9.199738,13.665898,2.147459,2.219132,0.826867,2.242558,1.600162e+04
min,119.000000,22.000000,8.928571,39.000000,1.000000,31.700000,33.000000,1.100000,320.000000,52.000000,5.150000,35.00000,21.000000,26.900000,11.100000,10.400000,1.000000,1.000000,4.000000,1.000000,31.000000,24.000000,0.200000,6.000000,0.020000,54.000000,23.000000,23.000000,37.000000,3.000000,0.900000,22.000000,1.000000,0.010000,2.000000,63.000000,70.000000,5.000000,9.000000,1.000000,0.200000,0.005000,0.100000,3.956543e+06
25%,166.500000,57.800000,21.500000,81.000000,1.000000,69.750000,102.500000,4.000000,5000.000000,437.000000,12.850000,40.95000,88.750000,29.900000,32.300000,21.700000,9.000000,4.000000,11.000000,15.000000,158.000000,164.000000,0.680000,11.400000,0.750000,71.475000,45.250000,54.000000,90.332500,6.700000,1.000000,50.000000,3.000000,0.039000,5.100000,86.000000,87.250000,15.575000,11.000000,1.000000,0.850000,0.810000,0.750000,4.012100e+06
50%,170.000000,64.000000,22.600000,84.900000,3.000000,80.000000,123.000000,5.000000,6100.000000,482.500000,14.450000,44.00000,92.000000,31.100000,33.100000,25.150000,20.500000,4.400000,20.000000,28.000000,173.000000,207.000000,0.850000,14.400000,0.870000,80.050000,57.500000,84.000000,116.000000,7.150000,1.600000,68.500000,5.750000,0.100000,5.400000,91.000000,92.000000,21.000000,15.400000,3.000000,1.000000,1.000000,1.000000,4.015656e+06
75%,173.100000,72.300000,24.950000,85.000000,4.120000,84.250000,130.000000,6.000000,7400.000000,514.250000,15.225000,46.20000,94.525000,32.100000,34.000000,30.750000,30.000000,4.975000,30.000000,50.000000,184.055000,274.000000,1.000000,17.100000,1.000000,86.600000,75.350000,147.500000,130.000000,7.300000,1.740000,78.750000,6.450000,0.162500,5.700000,99.000000,104.250000,26.975000,33.100000,4.500000,1.750000,1.287500,1.350000,4.032715e+06
max,190.000000,480.000000,57.803468,185.000000,5.150000,147.000000,170.000000,14.000000,10170.000000,1422.000000,30.000000,81.00000,219.000000,35.000000,54.000000,82.000000,61.000000,71.000000,76.000000,147.000000,252.000000,382.000000,5.000000,32.100000,3.000000,111.110000,193.000000,317.000000,208.000000,7.900000,2.300000,124.000000,12.000000,0.654000,7.600000,201.500000,183.000000,53.000000,51.900000,8.400000,9.000000,3.000000,12.000000,4.037169e+06


In [26]:
denied.describe()

,Height,Weight,BMI,AbdominalCircumference,VitalCapacity,BloodPressureSecond,BloodPressureFirst,UrinePH,Leukocyte,RedBloodCells,Hemoglobin,Hematocrit,MCV,MCH,MCHC,platelet,GOT,albumin,GPT,GammaGPT,LDH,ALP,Bilirubin,UreaNitrogen,creatinine,eGFR,HDL,NeutralFat,LDL,TotalProtein,AGRatio,Amylase,UricAcid,CRP,HbA1c,BloodSugar,FastingBloodSugar,bodyFatPercentage,Lym,TTT,RightEyeCorrection,Prostate,LeftEyeCorrection,PatientID
count,32.000000,24.000000,41.000000,19.000000,17.000000,11.000000,18.000000,28.000000,14.000000,36.000000,34.000000,36.000000,28.000000,33.000000,33.000000,33.000000,78.000000,29.000000,64.000000,73.000000,21.000000,29.000000,30.000000,4.000000,47.000000,18.000000,52.000000,33.000000,44.000000,13.000000,15.000000,15.000000,30.000000,19.000000,19.000000,23.000000,19.000000,10.000000,15.000000,9.000000,14.000000,35.000000,14.000000,1.780000e+02
mean,169.362500,83.179792,23.825852,87.021053,2.967059,73.454545,108.911111,5.539286,6040.000000,390.916667,12.064706,42.741667,93.107143,34.812121,31.366061,28.157576,25.048718,10.232069,21.265625,35.322507,182.571429,202.244828,1.317667,14.975000,0.789574,75.422222,68.115385,119.615152,119.727273,6.800000,1.621333,71.666667,4.373333,0.105947,6.011053,105.826087,98.873684,17.880000,22.786667,2.055556,2.128571,1.063000,1.957143,4.020214e+06
std,7.773352,67.457047,5.398528,17.492684,2.063067,8.201995,31.110768,2.181291,1515.392815,184.324772,3.638172,5.604660,11.654499,16.114430,4.836632,17.894396,20.117814,17.287567,14.329304,30.230007,53.305320,81.937492,0.908833,5.286067,0.179600,11.048905,30.769640,80.400832,39.419251,0.921954,0.733474,29.851218,2.792222,0.102891,1.790837,51.546671,26.296025,8.579666,16.265953,1.628735,1.884932,0.712109,3.157339,1.390724e+04
min,157.200000,22.000000,17.200000,35.300000,1.000000,57.000000,39.000000,3.000000,3800.000000,53.000000,5.500000,29.000000,76.000000,24.000000,12.780000,10.400000,1.000000,1.000000,4.000000,1.000000,45.000000,21.000000,0.200000,9.700000,0.300000,55.900000,31.000000,30.000000,58.000000,4.000000,1.000000,37.000000,1.000000,0.010000,3.000000,75.000000,36.000000,3.000000,8.100000,1.000000,0.300000,0.295000,0.100000,3.978258e+06
25%,161.875000,54.625000,20.100000,83.450000,1.000000,69.500000,85.525000,4.000000,5125.000000,300.500000,8.000000,40.675000,88.950000,29.000000,31.000000,21.000000,13.250000,4.000000,11.325000,13.000000,155.000000,152.000000,0.727500,11.800000,0.700000,67.700000,44.750000,61.000000,95.250000,6.800000,1.200000,51.500000,1.000000,0.031000,5.200000,88.500000,89.000000,10.900000,10.900000,1.000000,1.000000,0.590000,0.300000,4.013091e+06
50%,170.000000,65.850000,22.500000,85.000000,2.640000,76.000000,120.000000,5.250000,5950.000000,467.000000,13.200000,43.400000,90.350000,31.000000,32.700000,23.800000,21.500000,4.300000,18.000000,26.100000,183.000000,201.100000,1.000000,14.100000,0.780000,78.650000,61.500000,99.000000,118.000000,7.000000,1.500000,63.000000,5.200000,0.100000,5.600000,98.000000,99.000000,19.950000,18.900000,1.000000,1.600000,1.000000,1.000000,4.016826e+06
75%,175.025000,85.128750,26.000000,91.500000,4.280000,79.500000,129.750000,6.000000,6925.000000,515.250000,14.850000,45.925000,96.000000,33.200000,34.000000,27.000000,30.000000,4.600000,29.000000,50.000000,211.000000,229.000000,1.800000,17.275000,0.890000,81.950000,78.750000,167.000000,148.750000,7.200000,1.750000,80.000000,7.000000,0.120000,6.150000,108.000000,108.500000,23.550000,28.900000,2.000000,2.750000,1.131500,1.425000,4.032582e+06
max,183.600000,373.000000,39.843750,130.000000,7.000000,84.000000,162.000000,15.000000,9100.000000,642.000000,17.100000,57.000000,146.000000,101.000000,35.000000,85.000000,97.000000,65.700000,68.000000,132.000000,314.000000,397.000000,3.900000,22.000000,1.190000,99.000000,174.000000,311.000000,215.000000,7.600000,4.000000,134.000000,8.300000,0.330000,9.600000,336.000000,181.000000,28.000000,71.000000,5.500000,7.000000,3.000000,12.000000,4.037800e+06
